In [2]:
import requests
import datetime
from dateutil.relativedelta import relativedelta  # to add days or years
from datetime import date
import streamlit as st
import requests
import pandas as pd
import plotly.graph_objects as go
from bokeh.models.widgets import Div
import os
import requests
import json
import os
import sys
from collections import Counter
from os.path import dirname, exists, join
import gensim.downloader as api
import meilisearch
import nltk
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer

In [7]:
query = 'Technology'
params = dict(q=query)

today = datetime.datetime.now()
limit_date = today + relativedelta(days=-7)
today_time = today.timestamp()
limit_time = limit_date.timestamp()



tweet_params = dict(q=query,
                    filters=f"timestamp < {limit_time}",
                    limit=100000)
tweet_params_without_query = dict(q="",
                                  filters=f"timestamp < {limit_time}")

headers = {'X-Meili-API-Key': 'YjI1YzZhMmE4YTA0NmRjNTA5YTUxOTFi'}





def get_consultations():
    lexai_url = "http://35.225.139.215/indexes/consultations/search"
    result = requests.get(lexai_url, params=params, headers=headers).json()
    consultations = []
    for i in result["hits"]:
        title = i['title']
        topics = i['topics']
        type_of_act = i['type_of_act']
        try:
            status = i["status"]
        except:
            status= ""
        try:
            end_date = pd.to_datetime(i['end_date'],errors='coerce').date()
        except:
            end_date = pd.to_datetime(i['end_date'], errors='coerce')
        link = i['link']
        consultations.append({
            "title": title,
            "status": status,
            "topics": topics,
            "type_of_act": type_of_act,
            "end_date": end_date,
            "link": link
        })

    return consultations

In [10]:
lexai_url_news = "http://35.225.139.215/indexes/twitter_press/search"
news = requests.get(lexai_url_news, params=tweet_params,
                    headers=headers).json()

news

{'status': 'MeiliSearch is running'}

In [4]:
#Data from News
lexai_url_news = "http://35.225.139.215/indexes/twitter_press/search"
news = requests.get(lexai_url_news, params=tweet_params,
                    headers=headers).json()

#Data from Politicians
lexai_url_politicians = "http://127.0.0.1:7700/indexes/twitter_politicians/search"
politicians = requests.get(lexai_url_politicians,
                           params=tweet_params,
                           headers=headers).json()

#Data from General
lexai_url_general = f"http://127.0.0.1:7700/indexes/twitter_query/search/"
query_data_general = requests.get(lexai_url_general,
                                  params=tweet_params,
                                  headers=headers).json()

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=7700): Max retries exceeded with url: /indexes/twitter_politicians/search?q=Technology&filters=timestamp+%3C+1627293969.475802&limit=100000 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1472a8df0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [61]:

# Sentiment pie-charts
#All data
query_df = pd.DataFrame(query_data_general['hits'])
politicians_df = pd.DataFrame(politicians['hits'])
news_df = pd.DataFrame(news['hits'])
all_tweets_df = pd.concat([query_df, politicians_df, news_df])

colors_pie = ['#11E7FB', '#6082FD', '#731F7D']

labels = ["Positive", "Neutral", "Negative"]

values_query = [
    all_tweets_df['sentiment'].value_counts()['positive'],
    all_tweets_df['sentiment'].value_counts()['neutral'],
    all_tweets_df['sentiment'].value_counts()['negative']
]
values_all = [41, 35, 24]

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1,
                    cols=2,
                    specs=[[{
                        'type': 'domain'
                    }, {
                        'type': 'domain'
                    }]])
fig.add_trace(
    go.Pie(title='Twitter',
           labels=labels,
           values=values_all,
           direction='clockwise',
           sort=False), 1, 1)
fig.add_trace(
    go.Pie(title=query,
        labels=labels,
           values=values_query,
           direction='clockwise',
           sort=False), 1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4,
                  hoverinfo="label+percent",
                  textfont_size=15,
                  marker=dict(colors=colors_pie))

fig.update_layout(
    # Add annotations in the center of the donut pies.
    annotations=[
        dict(text='',
             x=0.18,
             y=0.5,
             font_size=18,
             font_color='#731F7D',
             showarrow=False),
        dict(text="",
             x=0.82,
             y=0.5,
             font_size=18,
             font_color='#731F7D',
             showarrow=False)], autosize=False, width=500, height=250, margin=dict(l=0,r=0,b=0,t=0,pad=1))


KeyError: 'sentiment'

In [62]:
all_tweets_df

,timestamp,id,user,text,text_en,date,lang,iso_lang,user_verified,followers_count,user_loc,user_desc,user_desc_en,user_image,hashtags,mentions,retweet_count,favorite_count,link
0,1.622783e+09,1400711160509341698,hanisshit,tongue technology,tongue technology,2021/06/04 07:08:59,cs,cs,False,8,,hani's shitpost acct,hani&#39;s shitpost acct,http://pbs.twimg.com/profile_images/7650178313...,,,0,0,https://twitter.com/hanisshit/status/140071116...
1,1.622783e+09,1400711066246545411,moothought,when is technology going to reach baby transla...,when is technology going to reach baby transla...,2021/06/04 07:08:36,en,en,False,438,Twitter-verse,( Insert Adjectives ),( Insert Adjectives ),http://pbs.twimg.com/profile_images/1375600698...,,,0,0,https://twitter.com/moothought/status/14007110...
2,1.622783e+09,1400711150413754370,scrumnl,RT @rob_england: @technology @emilychangtv Say...,RT @rob_england: @technology @emilychangtv Say...,2021/06/04 07:08:56,en,en,False,469,Room 3B,Rincewindian Post-Agilist,Rincewindian Post-Agilist,http://pbs.twimg.com/profile_images/1227853908...,,"rob_england, technology, emilychangtv",2,0,https://twitter.com/scrumnl/status/14007111504...
3,1.622783e+09,1400711119887646724,CantStopStompin,RT @veritasnewsfeed: mRNA technology pioneer s...,RT @veritasnewsfeed: mRNA technology pioneer s...,2021/06/04 07:08:49,en,en,False,18,,Never ever give up. 28. Gay. Trumplican.,Never ever give up. 28. Gay. Trumplican.,http://pbs.twimg.com/profile_images/1389693339...,,veritasnewsfeed,57,0,https://twitter.com/CantStopStompin/status/140...
4,1.622783e+09,1400711120067911683,hannahfrankbot,DOCTOR: Little bit of technology from my home.,DOCTOR: Little bit of technology from my home.,2021/06/04 07:08:49,en,en,False,0,,,None,http://abs.twimg.com/sticky/default_profile_im...,,,0,0,https://twitter.com/hannahfrankbot/status/1400...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,1.622690e+09,1400318866757128192,euobs,The partnership will target technologies with ...,None,2021/06/03 05:10:09,en,None,True,197551,Brussels,Influential. Investigative. Independent. EUobs...,None,http://pbs.twimg.com/profile_images/1345057594...,,,0,1,https://twitter.com/euobs/status/1400318866757...
7,1.622641e+09,1400112472984457223,euobs,The EU is falling behind the US and China in #...,None,2021/06/02 15:30:01,en,None,True,197551,Brussels,Influential. Investigative. Independent. EUobs...,None,http://pbs.twimg.com/profile_images/1345057594...,"ArtificialIntelligence, blockchain",EIB,0,1,https://twitter.com/euobs/status/1400112472984...
8,1.622721e+09,1400448440748945412,EURACTIV,.@EU_Commission announcement: Bill Gates-found...,None,2021/06/03 13:45:02,en,None,True,137848,Brussels,EU news & policy debates from the European Med...,None,http://pbs.twimg.com/profile_images/8235903410...,Climate,"EU_Commission, Breakthrough",0,6,https://twitter.com/EURACTIV/status/1400448440...
9,1.622617e+09,1400014358260953089,EURACTIV,🇫🇮 What will drive Helsinki's transition to #C...,🇫🇮 What will drive Helsinki's transition to #C...,2021/06/02 09:00:08,en,None,True,137846,Brussels,EU news & policy debates from the European Med...,EU news & policy debates from the European Med...,http://pbs.twimg.com/profile_images/8235903410...,CarbonNeutrality,FredSimonEU,4,3,https://twitter.com/EURACTIV/status/1400014358...


In [3]:
import json
import os
import sys
from collections import Counter
from os.path import dirname, exists, join

import gensim.downloader as api
import meilisearch
import nltk
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer

load_dotenv(dotenv_path=join(dirname(dirname(__file__)),'.env'))

class Analyse:
    
    def __init__(self, url='http://127.0.0.1:7700', key=None):
        
        if key is None:
            key = os.getenv('MEILISEARCH_KEY')
        
        self.client = meilisearch.Client(url, key)
        print
        self.indices = [idx.get('name', None) for idx in self.client.get_indexes()]
        
    @staticmethod
    def get_words():
        text_keys = {
            'eurlex': ['title'],
            'consultations': ['title', 'topics'],
            'twitter_query': ['text_en', 'user_desc_en'],
            'twitter_politicians': ['text_en', 'user_desc_en'],
            'twitter_press': ['text_en', 'user_desc_en']
        }
        
        print('Extracting all lemmatized words from json files... ', end='')
        folder = join(os.path.abspath(''), 'data.json')
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return

        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        if len(files) == 0:  # if no files
            print('No files found.')
            return

        words = []
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        tokenizer = nltk.RegexpTokenizer(r"\w+")

        for index, filename in zip(indices, files):
            with open(join(folder, filename), 'r') as file:
                json_ = json.load(file)

            for i in range(len(json_)):
                for key in text_keys[index]:
                    if json_[i][key] is not None:
                        text = json_[i][key].lower().replace(r'\d','')
                        tokens = [lemmatizer.lemmatize(word) for word in tokenizer.tokenize(text) if word not in stop_words]
                        words.extend(tokens)
        
        print('Done.\n')
        return words
    
    def get_synonyms(self, frac=0.03, n=0.5):
        words = self.get_words()
        
        print('Loading model... ', end='')
        word2vec_transfer = api.load('word2vec-google-news-300')
        print('Done.')
        
        print('Getting available words in model... ', end='')
        available_words = list(word2vec_transfer.key_to_index.keys())
        print('Done.')
        
        print('Extracting nouns and adjectives...', end='')
        words_to_keep = set([word for (word, tag) in nltk.pos_tag(words) if any(i in tag for i in ['NN', 'JJ'])])
        print('Done.')
        
        print('Filtering words... ', end='')
        word_counts = {k: v for k, v in Counter(words).items() if k in words_to_keep}
        if isinstance(frac, int):
            min_count = frac
        else:
            min_count = sorted(list(word_counts.values()))[::-1][int(len(word_counts.values()) * 0.03)]
        words = [word for word in words_to_keep if (word_counts[word] > min_count and word in available_words)]
        print('Done.')
        
        print('Finding synonyms... ', end='')
        if n >= 1:
            synonyms = {word: list(np.array(word2vec_transfer.most_similar(word, topn=n))[:,0]) for word in words}
        else:
            synonyms = {}
            for word in words:
                arr = np.array(word2vec_transfer.most_similar(word, topn=20))
                synonyms[word] = list(arr[arr[:,1].astype(float) > n][:,0])
        print('Done.')
        
        folder = join(os.path.abspath(''), 'data.json')
        with open(join(folder, 'synonyms.json'), 'w') as file:
            json.dump(synonyms, file)
        
        print('Saved synonyms to data.json/synonyms.json\n')
        return synonyms
    
    def get_all_sentiments(self):
        folder = dirname(dirname(__file__))
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return
        
        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        
        if len(files) == 0:  # if no files
            print('No files found.')
            return
        
        for index, filename in zip(indices, files):
            filepath = join(folder, 'data.json', filename)
            with open(filepath, 'r') as file:
                documents = json.load(file)
            
            filepath = join(folder, 'update.json', f'{index}_sentiment.json')
            with open(filepath, 'w') as file:
                json.dump(self.get_sentiments(documents), file)

    
    def get_sentiments(self, data):
        #initialize sentiment analyzer
        sid = SentimentIntensityAnalyzer()

        # make dict of scores
        sentiments = {i : {'id': doc['id'], 
                           'compound_score': sid.polarity_scores(doc['text_en'])['compound']}
                      for i, doc in data.items()}

        # add category
        for k, v in sentiments.items():
            if sentiments[k]['compound_score'] <= -0.2:
                sentiments[k]['sentiment'] = 'negative'
            elif sentiments[k]['compound_score'] > 0.2:
                sentiments[k]['sentiment'] = 'positive'
            else:
                sentiments[k]['sentiment'] = 'neutral'
            
        return sentiments

if __name__ == '__main__':
    if len(sys.argv) > 2:
        try:
            kwargs = {kwarg.split("=")[0]: eval(kwarg.split("=")[1]) for kwarg in sys.argv[2:]}
        except NameError:
            kwargs = {kwarg.split("=")[0]: kwarg.split("=")[1].split(",") for kwarg in sys.argv[2:]}
        getattr(Analyse(), sys.argv[1])(**kwargs)
    elif len(sys.argv) == 2:
        getattr(Analyse(), sys.argv[1])()


NameError: name '__file__' is not defined

In [4]:
def get_all_sentiments(self):
        folder = dirname(dirname(__file__))
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return
        
        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        
        if len(files) == 0:  # if no files
            print('No files found.')
            return
        
        for index, filename in zip(indices, files):
            filepath = join(folder, 'data.json', filename)
            with open(filepath, 'r') as file:
                documents = json.load(file)
            
            filepath = join(folder, 'update.json', f'{index}_sentiment.json')
            with open(filepath, 'w') as file:
                json.dump(self.get_sentiments(documents), file)

    
def get_sentiments(self, data):
    #initialize sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # make dict of scores
    sentiments = {i : {'id': doc['id'], 
                       'compound_score': sid.polarity_scores(doc['text_en'])['compound']}
                  for i, doc in data.items()}

    # add category
    for k, v in sentiments.items():
        if sentiments[k]['compound_score'] <= -0.2:
            sentiments[k]['sentiment'] = 'negative'
        elif sentiments[k]['compound_score'] > 0.2:
            sentiments[k]['sentiment'] = 'positive'
        else:
            sentiments[k]['sentiment'] = 'neutral'

    return sentiments



In [5]:
def extract_info(self, result, df):
        entry = {}
    
        entry['id'] = result['id']
        entry['user'] = result['user']['screen_name']
        text = normalize('NFKD', result['full_text'])
        entry['text'] = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)
        entry['text_en'] = self.gtrans(entry['text'], dest='en')
        
        dt = datetime.strptime(result['created_at'],
                                "%a %b %d %H:%M:%S %z %Y")
        entry['date'] = datetime.strftime(dt, "%Y/%m/%d %H:%M:%S")
        entry['lang'] = result['lang']
        entry['iso_lang'] = result.get('metadata', {}).get('iso_language_code', None)
        
        entry['user_verified'] = result['user']['verified']
        entry['followers_count'] = result['user']['followers_count']
        entry['user_loc'] =  normalize('NFKD', result['user']['location'])
        entry['user_desc'] = normalize('NFKD', result['user']['description'])
        entry['user_desc_en'] = self.gtrans(entry['user_desc'], dest='en')
        entry['user_image'] = result['user']['profile_image_url']
        
        entry['hashtags'] = ', '.join([i['text'] 
                                        for i in result['entities']['hashtags']])
        entry['mentions'] = ', '.join([i['screen_name'] 
                                        for i in result['entities']['user_mentions']])
        entry['retweet_count'] = result['retweet_count']
        entry['favorite_count'] = result['favorite_count']
        
        entry['timestamp'] = mktime(dt.timetuple())
        entry['link'] = f"https://twitter.com/{entry['user']}/status/{entry['id']}"

        user = entry['user'].lower().strip()
        if all(col in df.columns for col in ['name', 'country', 'eu_group', 'national_group']):
            entry['name'] = df[df['twitter'].str.lower() == user]['name'].iloc[0]
            entry['country'] = df[df['twitter'].str.lower() == user]['country'].iloc[0]
            entry['eu_group'] = df[df['twitter'].str.lower() == user]['eu_group'].iloc[0]
            entry['national_group'] = df[df['twitter'].str.lower() == user]['national_group'].iloc[0]
        elif 'media' in df.columns:
            entry['name'] = df[df['twitter'].str.lower() == user]['media'].iloc[0]
            entry['country'] = df[df['twitter'].str.lower() == user]['country'].iloc[0]
        
        if entry['text_en'] is not None:
            sid = SentimentIntensityAnalyzer()
            entry['compound_score'] = sid.polarity_scores(entry['text_en'])['compound']
            if entry['compound_score'] <= -0.2:
                entry['sentiment'] = 'negative'
            elif entry['compound_score'] > 0.2:
                entry['sentiment'] = 'positive'
            else:
                entry['sentiment'] = 'neutral'
        
        return entry

In [6]:
url="http://localhost:7700/indexes"
result = requests.get(url, params=params, headers=headers).json()

NameError: name 'params' is not defined

In [12]:
def get_all_sentiments():
        folder = dirname(dirname(os.getcwd()))+"/LexAI"
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return
        
        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        
        if len(files) == 0:  # if no files
            print('No files found.')
            return
        
        for index, filename in zip(indices, files):
            filepath = join(folder, 'data.json', filename)
            with open(filepath, 'r') as file:
                documents = json.load(file)
            
            filepath = join(folder, 'update.json', f'{index}_sentiment.json')
            with open(filepath, 'w') as file:
                json.dump(self.get_sentiments(documents), file)


In [13]:
get_all_sentiments()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/estefaniavidalbouzon/code/EstefaniaVB/LexAI/LexAI/data.json/setup.sh'

In [14]:
files = os.listdir(folder)
indices = [file.replace('.json', '') for file in files]

NameError: name 'folder' is not defined

In [15]:
files, indices

NameError: name 'files' is not defined

In [25]:
def get_all_sentiments():
    folder = os.getcwd()
    if not exists(folder):  # if folder doesn't exist
        print('Folder not found.')
        return

    files = os.listdir(os.getcwd()+"/data.json")
    indices = [file.replace('.json', '') for file in files]

    if len(files) == 0:  # if no files
        print('No files found.')
        return

    for index, filename in zip(indices, files):
        filepath = join(folder, 'data.json', filename)
        with open(filepath, 'r') as file:
            documents = json.load(file)

        filepath = join(folder, 'update.json', f'{index}_sentiment.json')
        with open(filepath, 'w') as file:
            json.dump(self.get_sentiments(documents), file)

    
def get_sentiments(data):
    #initialize sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # make dict of scores
    sentiments={}

    for n,i in enumerate(data):
        try :
            sentiments[n]={"id":i["id"],"compound_score":sid.polarity_scores(i['text_en'])['compound']}
        except:
            pass
            #sentiments[n]={"id":i["id"],"compound_score":"NaN"}
            #print(n, sentiments[n], i["id"])


    # add category
    for k, v in sentiments.items():
        if sentiments[k]['compound_score'] <= -0.2:
            sentiments[k]['sentiment'] = 'negative'
        elif sentiments[k]['compound_score'] > 0.2:
            sentiments[k]['sentiment'] = 'positive'
        else:
            sentiments[k]['sentiment'] = 'neutral'

    return sentiments

In [26]:
folder = dirname(os.getcwd())
print(folder)

files = os.listdir(dirname(os.getcwd())+"/data.json")
print(files)
indices = [file.replace('.json', '') for file in files]
print(indices)

for index, filename in zip(indices, files):
    print(index,filename)
    print("index")
    filepath = join(folder, 'data.json', filename)
    with open(filepath, 'r') as file:
        documents = json.load(file)
        print(len(documents))
    filepath = join(folder, 'update.json', f'{index}_sentiment.json')
    print("senti")
    with open(filepath, 'w') as file:
        json.dump(get_sentiments(data= documents), file)


/Users/estefaniavidalbouzon/code/EstefaniaVB/LexAI/LexAI
['twitter_query.json', 'eurlex.json', 'consultations.json', 'twitter_press.json', 'twitter_politicians.json']
['twitter_query', 'eurlex', 'consultations', 'twitter_press', 'twitter_politicians']
twitter_query twitter_query.json
index
17807
senti
eurlex eurlex.json
index
4962
senti
consultations consultations.json
index
920
senti
twitter_press twitter_press.json
index
6147
senti
twitter_politicians twitter_politicians.json
index
27369
senti


In [27]:
def extract_info(self, result, df):
        entry = {}
    
        entry['id'] = result['id']
        entry['user'] = result['user']['screen_name']
        text = normalize('NFKD', result['full_text'])
        entry['text'] = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)
        entry['text_en'] = self.gtrans(entry['text'], dest='en')
        
        dt = datetime.strptime(result['created_at'],
                                "%a %b %d %H:%M:%S %z %Y")
        entry['date'] = datetime.strftime(dt, "%Y/%m/%d %H:%M:%S")
        entry['lang'] = result['lang']
        entry['iso_lang'] = result.get('metadata', {}).get('iso_language_code', None)
        
        entry['user_verified'] = result['user']['verified']
        entry['followers_count'] = result['user']['followers_count']
        entry['user_loc'] =  normalize('NFKD', result['user']['location'])
        entry['user_desc'] = normalize('NFKD', result['user']['description'])
        entry['user_desc_en'] = self.gtrans(entry['user_desc'], dest='en')
        entry['user_image'] = result['user']['profile_image_url']
        
        entry['hashtags'] = ', '.join([i['text'] 
                                        for i in result['entities']['hashtags']])
        entry['mentions'] = ', '.join([i['screen_name'] 
                                        for i in result['entities']['user_mentions']])
        entry['retweet_count'] = result['retweet_count']
        entry['favorite_count'] = result['favorite_count']
        
        entry['timestamp'] = mktime(dt.timetuple())
        entry['link'] = f"https://twitter.com/{entry['user']}/status/{entry['id']}"

        user = entry['user'].lower().strip()
        if all(col in df.columns for col in ['name', 'country', 'eu_group', 'national_group']):
            entry['name'] = df[df['twitter'].str.lower() == user]['name'].iloc[0]
            entry['country'] = df[df['twitter'].str.lower() == user]['country'].iloc[0]
            entry['eu_group'] = df[df['twitter'].str.lower() == user]['eu_group'].iloc[0]
            entry['national_group'] = df[df['twitter'].str.lower() == user]['national_group'].iloc[0]
        elif 'media' in df.columns:
            entry['name'] = df[df['twitter'].str.lower() == user]['media'].iloc[0]
            entry['country'] = df[df['twitter'].str.lower() == user]['country'].iloc[0]
        
        if entry['text_en'] is not None:
            sid = SentimentIntensityAnalyzer()
            entry['compound_score'] = sid.polarity_scores(entry['text_en'])['compound']
            if entry['compound_score'] <= -0.2:
                entry['sentiment'] = 'negative'
            elif entry['compound_score'] > 0.2:
                entry['sentiment'] = 'positive'
            else:
                entry['sentiment'] = 'neutral'
        
        return entry

In [70]:
files = os.listdir(dirname(os.getcwd())+"/data.json")
filepath = join(folder, 'data.json', filename)
with open(filepath, 'r') as file:
    twitter_query = json.load(file)

In [71]:
twitter_query

[{'id': 1400117440827342848,
  'timestamp': 1622641785.0,
  'user': 'PalmerReport',
  'text': 'Arizona’s Democratic Secretary of State Katie Hobbs is running for Governor of Arizona in 2022.\n\nFlorida’s Democratic Commissioner of Agriculture Nikki Fried is running for Governor of Florida in 2022.\n\nStrong women are taking over. You love to see it.',
  'text_en': 'Arizona’s Democratic Secretary of State Katie Hobbs is running for Governor of Arizona in 2022.\n\nFlorida’s Democratic Commissioner of Agriculture Nikki Fried is running for Governor of Florida in 2022.\n\nStrong women are taking over. You love to see it.',
  'date': '2021/06/02 15:49:45',
  'lang': 'en',
  'iso_lang': 'en',
  'user_verified': True,
  'followers_count': 383615,
  'user_loc': 'Los Angeles, CA',
  'user_desc': 'Political analysis ahead of the curve.',
  'user_desc_en': 'Political analysis ahead of the curve.',
  'user_image': 'http://pbs.twimg.com/profile_images/1301392629230338048/wEJnZzSn_normal.jpg',
  'ha

In [75]:
def get_sentiments(data):
    #initialize sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # make dict of scores
    sentiments = {i : {'id': i['id'], 
                       'compound_score': sid.polarity_scores(i['text_en'])['compound']}
                  for e in data for i, doc in e.items()}

    # add category
    for k, v in sentiments.items():
        if sentiments[k]['compound_score'] <= -0.2:
            sentiments[k]['sentiment'] = 'negative'
        elif sentiments[k]['compound_score'] > 0.2:
            sentiments[k]['sentiment'] = 'positive'
        else:
            sentiments[k]['sentiment'] = 'neutral'

    return sentiments


In [81]:
def get_all_sentiments():
    folder = dirname(os.getcwd())
    if not exists(folder):  # if folder doesn't exist
        print('Folder not found.')
        return

    files = os.listdir(folder)
    indices = [file.replace('.json', '') for file in files]

    if len(files) == 0:  # if no files
        print('No files found.')
        return

    for index, filename in zip(indices, files):
        filepath = join(folder, 'data.json', filename)
        with open(filepath, 'r') as file:
            documents = json.load(file)

        filepath = join(folder, 'update.json', f'{index}_sentiment.json')
        with open(filepath, 'w') as file:
            json.dump(get_sentiments(documents), file)

def get_sentiments(data):
    #initialize sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # make dict of scores
    sentiments = {i : {'id': doc['id'], 
                       'compound_score': sid.polarity_scores(doc['text_en'])} 
                  for i, doc in data.items()}

    # add category
    for k, v in sentiments.items():
        if sentiments[k]['compound_score'] <= -0.2:
            sentiments[k]['sentiment'] = 'negative'
        elif sentiments[k]['compound_score'] > 0.2:
            sentiments[k]['sentiment'] = 'positive'
        else:
            sentiments[k]['sentiment'] = 'neutral'

    return sentiments

In [82]:
get_all_sentiments()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/estefaniavidalbouzon/code/EstefaniaVB/LexAI/LexAI/data.json/setup.sh'

In [184]:
def get_all_sentiments():
        folder = dirname(dirname(os.getcwd()))+"/update.json"
        if not exists(folder):  # if folder doesn't exist
            print('Folder not found.')
            return
        
        files = os.listdir(folder)
        indices = [file.replace('.json', '') for file in files]
        
        if len(files) == 0:  # if no files
            print('No files found.')
            return
        
        for index, filename in zip(indices, files):
            filepath = join(folder, 'data.json', filename)
            with open(filepath, 'r') as file:
                documents = json.load(file)
            
            filepath = join(folder, 'update.json', f'{index}_sentiment.json')
            with open(filepath, 'w') as file:
                json.dump(self.get_sentiments(documents), file)

In [23]:
folder = dirname(os.getcwd())
if not exists(folder):  # if folder doesn't exist
    print('Folder not found.')
    

files = os.listdir(folder)
indices = [file.replace('.json', '') for file in files]

if len(files) == 0:  # if no files
    print('No files found.')
    

for index, filename in zip(indices, files):
    filepath = join(folder, 'data.json', filename)
    with open(filepath, 'r') as file:
        documents = json.load(file)

    filepath = join(folder, 'update.json', f'{index}_sentiment.json')
    with open(filepath, 'w') as file:
        json.dump(self.get_sentiments(documents), file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/estefaniavidalbouzon/code/EstefaniaVB/LexAI/LexAI/data.json/setup.sh'